<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Before-you-start" data-toc-modified-id="Before-you-start-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Before you start</a></span></li><li><span><a href="#Authentication-setup" data-toc-modified-id="Authentication-setup-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Authentication setup</a></span></li><li><span><a href="#Hands-off-workflow" data-toc-modified-id="Hands-off-workflow-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Hands-off workflow</a></span></li></ul></div>

# Analysis Ready Data (ARD) workflow for MODIS Aqua L2P SST collection

This notebook demonstrates how to created a gridded "Data Cube", essentialy an ARD, from native Level 2P sea surface temperature (SST) data from the MODIS Aqua (https://doi.org/10.5067/GHMDA-2PJ19) collection or dataset.  It can also be applied to Terra L2P SST and other similar L2 satellite collections.


## Before you start

Before you beginning this tutorial, make sure you have an Earthdata account: [https://urs.earthdata.nasa.gov](https://urs.earthdata.nasa.gov) for the operations envionrment (most common) or [https://uat.urs.earthdata.nasa.gov](https://uat.urs.earthdata.nasa.gov) for the UAT environment.

Accounts are free to create and take just a moment to set up.

## Authentication setup

We need some boilerplate up front to log in to Earthdata Login.  The function below will allow Python
scripts to log into any Earthdata Login application programmatically.  To avoid being prompted for
credentials every time you run and also allow clients such as curl to log in, you can add the following
to a `.netrc` (`_netrc` on Windows) file in your home directory:

```
machine uat.urs.earthdata.nasa.gov
    login <your username>
    password <your password>
```

Make sure that this file is only readable by the current user or you will receive an error stating
"netrc access too permissive."

`$ chmod 0600 ~/.netrc` 

*You'll need to authenticate using the netrc method when running from command line with [`papermill`](https://papermill.readthedocs.io/en/latest/). You can log in manually by executing the cell below when running in the notebook client in your browser.*

In [ ]:
from urllib import request, parse
from http.cookiejar import CookieJar
import getpass
import netrc
import requests
import urllib


def setup_earthdata_login_auth(endpoint):
    """
    Set up the request library so that it authenticates against the given Earthdata Login
    endpoint and is able to track cookies between requests.  This looks in the .netrc file 
    first and if no credentials are found, it prompts for them.

    Valid endpoints include:
        uat.urs.earthdata.nasa.gov - Earthdata Login UAT (Harmony's current default)
        urs.earthdata.nasa.gov - Earthdata Login production
    """
    try:
        username, _, password = netrc.netrc().authenticators(endpoint)
    except (FileNotFoundError, TypeError):
        # FileNotFound = There's no .netrc file
        # TypeError = The endpoint isn't in the netrc file, causing the above to try unpacking None
        print('Please provide your Earthdata Login credentials to allow data access')
        print('Your credentials will only be passed to %s and will not be exposed in Jupyter' % (endpoint))
        username = input('Username:')
        password = getpass.getpass()

    manager = request.HTTPPasswordMgrWithDefaultRealm()
    manager.add_password(None, endpoint, username, password)
    auth = request.HTTPBasicAuthHandler(manager)

    jar = CookieJar()
    processor = request.HTTPCookieProcessor(jar)
    opener = request.build_opener(auth, processor)
    request.install_opener(opener)

In [ ]:
setup_earthdata_login_auth('uat.urs.earthdata.nasa.gov')
#setup_earthdata_login_auth('uat.earthdata.nasa.gov')

# ARD workflow

The idea of this workflow is to:
1) Discover MODIS granules within a region of interest (ROI)

2) Make spatial subsets of the dicovered granules

3) Remap those subsets to a common grid

4) Aggregate the maps to a "Data Cube": a co-registed set of images aggregated into a single file the cover the ROI

This is done using a combination NASA earthdata search and harmony services, and the python modules for the GDAL image transformation software, and the NCO toolkit. 

The first code block sets the input datasets, and time and space bounds 

In [ ]:
# GHRSST MODIS Aqua L2P SST v2019.0 collection concept-id (UAT)
ccid = "C1234724470-POCLOUD"

# GHRSST MODIS Aqua L2P SST v2019.0 collection concept-id (CMR)
#ccid = "C1940473819-POCLOUD"

# GHRSST MODIS Terra L2P SST v2019.0 collection concept-id
#ccid = "C1234724471-POCLOUD"

# set the time  bounds  for search
start_time = '2020-07-01T00:01:15Z'
stop_time = '2020-07-30T00:01:15Z'
temporal_coverage = start_time + ',' + stop_time
print(temporal_coverage)

# set the space bounds
# mid Atlantic region
#west = -35. ; south = -5. ; east = -25. ; north = 5.

# Hawaiian Is.
west = -163 ; south = 15 ; east = -153 ; north = 25

# Southern California Bight
#west = -118 ; south = 32.5 ; east = -117 ; north = 33.5

spatial_coverage =  str(west) + ',' + str(south) + ',' + str(east) + ',' + str(north)
print(spatial_coverage)

# the local download directory
download_dir = "/Users/earmstro/Downloads/modis"


Kewyord parameters for the spatial and temporal granule CMR Harmony search

In [ ]:
params = {
    'scroll': "true",
    'page_size': 2000,
    'sort_key': "-start_date",
    'collection_concept_id': ccid, 
    'temporal': temporal_coverage,
    'bounding_box': spatial_coverage
}

params

Encode query parameters and create the complete search url:

In [ ]:
from os import makedirs
from os.path import isdir, basename
from urllib.parse import urlencode
from urllib.request import urlopen, urlretrieve
from datetime import datetime, timedelta
from json import dumps, loads

cmr = "cmr.uat.earthdata.nasa.gov"
#cmr = "cmr.earthdata.nasa.gov"

query = urlencode(params)
url = f"https://{cmr}/search/granules.umm_json?{query}"
print(url)

Print number of granules found in search

In [ ]:
with urlopen(url) as f:
    results = loads(f.read().decode())

print(f"{results['hits']}  granules found in search for collection (dataset): {ccid} ")

Neatly print the first granule record (if one was returned).  "concept-id" is the file (granule) reference we will use as input to Harmony subsetting service:

In [ ]:
if len(results['items'])>0:
    print(dumps(results['items'][0], indent=2))

Find the concept-id for each file.  This is the key input for the harmony subsetting API and operation.

In [ ]:
# Dump the granule concept Ids
file_ids = [r['meta']['concept-id'] for r in results['items']]
file_ids



Dump the corresponding filenames and location URLs for each concept id for inspection and well being

In [ ]:
# Dump the URLs (containing orignal filenames) for the corresponding files ids

file_urls = [[u['URL'] for u in r['umm']['RelatedUrls'] if u['Type']=="GET DATA"][0] for r in results['items']]
file_urls 

**Perform the subsetting operation using the Earthdata Harmony service**

In [ ]:
import shutil

# Harmony root
harmony_root = 'https://harmony.uat.earthdata.nasa.gov'

# Loop over the file (granule) ids
for i in range(len(file_urls)):
   try:  
        harmonyConfig = {
            'collection_id': ccid,     
            'ogc-api-coverages_version': '1.0.0',
            'variable': 'all',
            'granuleId': file_ids[i],
            'lat': '(' + str(south) + ":" + str(north) + ')',
            'lon': '(' + str(west) + ":" + str(east) + ')',        
            #'subsettingCrs': l2proj_encode,
            #'outputCrs': l2proj_encode,
            #'interpolation': 'near',
            #'width': 1000,
            #'height': 1000
        }
         
        # reproject granule    
        #harmony_url = harmony_root + '/{collection_id}/ogc-api-coverages/{ogc-api-coverages_version}/collections/{variable}/coverage/rangeset?granuleId={granuleId}&interpolation={interpolation}&outputCrs={outputCrs}&height={height}&width={width}'.format(**harmonyConfig)
        
        # subset granule
        harmony_url = harmony_root + '/{collection_id}/ogc-api-coverages/{ogc-api-coverages_version}/collections/{variable}/coverage/rangeset?granuleId={granuleId}&subset=lat{lat}&subset=lon{lon}'.format(**harmonyConfig)
        print('Harmony URL', harmony_url)
        
        # store output in a defined directory with a meaninful filename based on orginal name
        out_file = download_dir + 'subset_' + basename(file_urls[i])
       
        
        # -- execute the request --
        with request.urlopen(harmony_url) as response, open(out_file, 'wb') as output:
           shutil.copyfileobj(response, output) 
        
        
   except urllib.error.HTTPError as e:
        print(f"    [{datetime.now()}] FAILURE: {f}\n\n{e}\n{e.read()}\n")
        raise e                     
   except Exception as e:
        print(f"    [{datetime.now()}] FAILURE: {f}\n\n{e}\n")
        raise e
   else:
        print(f"    [{datetime.now()}]  SUCCESS for: {out_file}")
        print('    Content Size (bytes):', response.headers['Content-length'])
        print()

**Perform resampling/reprojection on subsets using the GDAL module and gdal.Warp().** GDAL will only work on one variable ("layer") at a time and also strip out important CF metadata and coordinate variables. Therefore we will use **NCO tricks** to correct these artifacts here and in the next steps. Use the 'pynco' module for NCO python bindings.

In [ ]:
from osgeo import gdal, gdalconst
from nco import Nco

nco = Nco()

# keyword args for gdal.Warp()
kwargs = {'format': 'netCDF', 'copyMetadata': True, 
           'outputBounds': [west, south, east, north], 
           'xRes': 0.01,
           'yRes' : 0.01,
           'dstSRS':'+proj=longlat +datum=WGS84 +no_defs',
         }
print(kwargs)

nc_vars = ['sea_surface_temperature', 'quality_level']

# Loop through subsetted files (use file_urls as the loop list) and warp into defined region from kwargs{}

for i in range(len(file_urls)):
    for j in range(len(nc_vars)): 
      variable = nc_vars[j]
     
      # input filename
      src_filename = download_dir + 'subset_' + basename(file_urls[i]) 
      print("source filename: ", src_filename)
        
      # load the netCDF 'layer' like sea_surface_temperature (variable)
      nc_file = 'NETCDF:' + src_filename + ':' + variable
      print(nc_file)
    
      # try/catch for GDAL steps. Dont work on empty netCDF file from subsetting operation  
      try:
        src = gdal.Open(nc_file, gdalconst.GA_ReadOnly)

        #subDatasets = src.GetSubDatasets()
        #print('subsdatatsets', subDatasets)

        # set output filename
        out_filename = download_dir + 'subset_reproject-' + variable +  '-' + basename(file_urls[i]) 
        ds = gdal.Warp(out_filename, src, **kwargs)
        print("")

        del ds
        del src     
        
        # add time dimenson to 'Band1' using NCO
        nco.ncecat(input=out_filename, output='tmp.nc', options=['-v Band1 -u time'])
        nco.ncks(input='tmp.nc', output=out_filename, options=['-v Band1'])

        # use NCO to copy (append with -A ) the time variable to the output and make it a record dimension
        nco.ncks(input=src_filename, output=out_filename, options=['-v time -A --mk_rec_dmn time'])

      except Exception as e:
        print(f"    [{datetime.now()}] FAILURE: \n")
        #raise e   
      else:
        print(f"    [{datetime.now()}]  SUCCESS for: {out_filename}")
        print()


**Add variable level CF metadata information that GDAL stripped out using NCO commands.** Use the 'pynco' module for NCO python bindings.

In [ ]:
import glob, os
#import shutil
from nco import Nco

nco = Nco()

sstConfig = {
            'scale_factor': 0.005,     
            'add_offset': 273.15,
            'valid_max': 10000,
            'valid_min': -1000,
            'long_name': 'sea surface temperature',
            'standard_name': 'sea_surface_skin_temperature',
            'coverage_content_type': 'physicalMeasurement'
            }

qualityConfig = {
            'valid_max': 0,
            'valid_min': 5, 
            'flag_values': '0b, 1b, 2b, 3b, 4b, 5b',
            'flag_meanings': 'no_data bad_data worst_quality low_quality acceptable_quality best_quality',
            'long_name': 'quality level of SST pixel',
            'coverage_content_type': 'qualityInformation'
            }
 
# Atted() method appears broken    
#sstOpts = [
#             c.Atted(mode="o", attName="scale_factor", varName=variable, Value=0.005, sType='f'), 
#             c.Atted(mode="o", attName="add_offsdt", varName=variable, Value=273.15, sType='f'), 
#             c.Atted(mode="o", attName="valid_min", varName=variable, Value=-1000, sType='s'), 
#]
            
nc_vars = ['sea_surface_temperature', 'quality_level']
os.chdir(download_dir)

for i in range(len(nc_vars)): 
    variable = nc_vars[i]
    reg_ex = "subset*" + variable + "*"
    for file in glob.glob(reg_ex):
        if variable == 'sea_surface_temperature':
          print(" -> Updating "+ file)
          nco.ncrename(input=file, output=file, options=['-v .Band1,sea_surface_temperature'])
        
          #sstOpts = {'mode':'o', 'attName':'scale_factor', 'varName':variable, 'Value':0.005, 'sType':'f'}
          #nco.ncatted(input=file, output=file, options=sstOpts)

          # update the SST variable attributes
          nco.ncatted(input=file, output=file, options=['-a scale_factor,'+ variable + ',o,f,{scale_factor}'.format(**sstConfig)])
          nco.ncatted(input=file, output=file, options=['-a add_offset,'+ variable + ',o,f,{add_offset}'.format(**sstConfig)])
          nco.ncatted(input=file, output=file, options=['-a valid_min,'+ variable + ',o,s,{valid_min}'.format(**sstConfig)])
          nco.ncatted(input=file, output=file, options=['-a valid_max,'+ variable + ',o,s,{valid_max}'.format(**sstConfig)])
          nco.ncatted(input=file, output=file, options=['-a long_name,'+ variable + ',o,c,"{long_name}"'.format(**sstConfig)])
          nco.ncatted(input=file, output=file, options=['-a standard_name,'+ variable + ',o,c,{standard_name}'.format(**sstConfig)])
          nco.ncatted(input=file, output=file, options=['-a coverage_content_type,'+ variable + ',o,c,{coverage_content_type}'.format(**sstConfig)])
            
        elif variable == 'quality_level':
          print(" -> Updating "+ file)
        
          nco.ncrename(input=file, output=file, options=['-v .Band1,' + variable])

          # update the quality variable attributes
          nco.ncatted(input=file, output=file, options=['-a valid_min,'+ variable + ',o,s,{valid_min}'.format(**qualityConfig)])
          nco.ncatted(input=file, output=file, options=['-a valid_max,'+ variable + ',o,s,{valid_max}'.format(**qualityConfig)])
          nco.ncatted(input=file, output=file, options=['-a long_name,'+ variable + ',o,c,"{long_name}"'.format(**qualityConfig)])
          nco.ncatted(input=file, output=file, options=['-a flag_values,'+ variable + ',o,c,"{flag_values}"'.format(**qualityConfig)])
          nco.ncatted(input=file, output=file, options=['-a flag_meanings,'+ variable + ',o,c,"{flag_meanings}"'.format(**qualityConfig)])
          nco.ncatted(input=file, output=file, options=['-a coverage_content_type,'+ variable + ',o,c,{coverage_content_type}'.format(**qualityConfig)])
     
    
print( "-Done-\n")

**Create the MODIS SST Data Cube.** Copy variable(s) to SST target files and catenate all of them into a final output  netCDF file using NCO.

In [ ]:
import glob, os
from nco import Nco

nco = Nco()
os.chdir(download_dir)


# Loop through the SST files and add variable quality_level to SST file (target)
reg_ex = "subset_reproject-sea_surface_temperature*"

print( "Copying quality_level variables to target sst files . . .")
for sst_file in glob.glob(reg_ex):    
    quality_file = sst_file.replace("sea_surface_temperature", "quality_level")  
    nco.ncks(input=quality_file, output=sst_file, options=['-v quality_level -A'])
    
    
# Create the data cube using NCO ncrcat command
print(". . . -Done- \n\nCreating MODIS SST data cube . . . ")
nco.ncrcat(input=glob.glob(reg_ex), output='MODIS_SST.data-cube.nc')
print(". . . -Done- \n")



Read the data with xarray and perform some plotting

In [ ]:
import xarray as xr
import matplotlib.pyplot as plt
%matplotlib inline

dataCube = download_dir + 'MODIS_SST.data-cube.nc'
xds = xr.open_dataset(dataCube)
#print(xds)

# create objects for subplots
fig, axes = plt.subplots(ncols=2)

# plot a time series at a specific location
xds.sea_surface_temperature.isel(lat = 200, lon = 300).plot(ax=axes[0], marker = '.', linestyle = 'None')

# a histogram of all points in region of interest
xds.sea_surface_temperature.plot(ax=axes[1])

plt.tight_layout()
plt.draw()


# Filter the dataset using quality information (quality_level value 4 and 5 are best data)
qc_dataset = xds.where((xds['sea_surface_temperature'] < 310) & (xds['quality_level'] >= 4))

# re plot the time series at a specific location and the regional histogram
fig, axes = plt.subplots(ncols=2)
qc_dataset.sea_surface_temperature.isel(lat = 200, lon = 300).plot(ax=axes[0], marker = '.', linestyle = 'None')
qc_dataset.sea_surface_temperature.plot(ax=axes[1])

plt.tight_layout()
plt.draw()